# < LeNet-5 >

![LeNet_Original_Image_48T74Lc.jpg](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)

## - Import libaries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np

## - Set GPU Envrionment

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## - Set Hyper-parameters
  

In [3]:
# For Training
EPOCHS     = 10
BATCH_SIZE = 256

# For Optimizer
LR         = 0.001
MOMENTUM   = 0.9

## - Prepare the Dataset

https://www.cs.toronto.edu/~kriz/cifar.html

### - Calculate Mean and Std

In [4]:
train_CIFAR10_raw = datasets.CIFAR10(root='./dataset/CIFAR10', train=True, download=True, transform=transforms.ToTensor())
imgs = [item[0] for item in train_CIFAR10_raw] # item[0] and item[1] are image and its label
imgs = torch.stack(imgs, dim=0).numpy()
print(imgs.shape)

# calculate mean over each channel (r,g,b)
mean_r = imgs[:,0].mean()
mean_g = imgs[:,1].mean()
mean_b = imgs[:,2].mean()
print(mean_r,mean_g,mean_b)

# calculate std over each channel (r,g,b)
std_r = imgs[:,0].std()
std_g = imgs[:,1].std()
std_b = imgs[:,2].std()
print(std_r,std_g,std_b)

# calculate mean_std over each channel std (r,g,b)
std_all = np.array([np.std(x, axis=(1,2)) for x in imgs])
std_r = std_all[:,0].mean()
std_g = std_all[:,1].mean()
std_b = std_all[:,2].mean()
print(std_r,std_g,std_b)


Files already downloaded and verified
(50000, 3, 32, 32)
0.49139968 0.48215827 0.44653124
0.24703233 0.24348505 0.26158768
0.20220213 0.19931543 0.20086348


### - Compare a Image Before/After Normalization

In [5]:
transform_CIFAR10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_CIFAR10_normalized = datasets.CIFAR10(root='./dataset/CIFAR10', train=True, download=True, transform=transform_CIFAR10)

print('Before Normalization: ', train_CIFAR10_raw[0][0].detach().numpy()[0,0,:5])
print('After Normalization: ', train_CIFAR10_normalized[0][0].detach().numpy()[0,0,:5])

Files already downloaded and verified
Before Normalization:  [0.23137255 0.16862746 0.19607843 0.26666668 0.38431373]
After Normalization:  [-1.2853558 -1.5955144 -1.45982   -1.1108913 -0.5293439]


### - DataLoader for Train / Test

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root      = './dataset/CIFAR10', 
        train     = True,
        download  = True,
        transform = transform_CIFAR10),
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory = True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root      = './dataset/CIFAR10', 
        train     = False,
        download  = True,
        transform = transform_CIFAR10),
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory = True,
    num_workers=4
)

Files already downloaded and verified
Files already downloaded and verified


## - Make the Model (LeNet-5)

![LeNet_Original_Image_48T74Lc.jpg](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)

- All the layer have tanh activaiton.
- C1 and C2 are 5 X 5 convolution.

In [7]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        ###############################
        #                             #
        #  Please, complete the code  #
        #                             #
        ###############################

    def forward(self, x):
        ###############################
        #                             #
        #  Please, complete the code  #
        #                             #
        ###############################
        return x

In [8]:
model     = LeNet5().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         AvgPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         AvgPool2d-4             [-1, 16, 5, 5]               0
            Conv2d-5            [-1, 120, 1, 1]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


c:\Users\NullA\realanaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [9]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## - Train the Model

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306708
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.297889
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.298924
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.297415
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.284962
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.292728
Train Epoch: 1 [30720/50000 (61%)]	Loss: 2.273861
Train Epoch: 1 [35840/50000 (71%)]	Loss: 2.274694
Train Epoch: 1 [40960/50000 (82%)]	Loss: 2.269002
Train Epoch: 1 [46080/50000 (92%)]	Loss: 2.257367
[1] Test Loss: 2.2510, Accuracy: 21.27%
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.248024
Train Epoch: 2 [5120/50000 (10%)]	Loss: 2.232392
Train Epoch: 2 [10240/50000 (20%)]	Loss: 2.211924
Train Epoch: 2 [15360/50000 (31%)]	Loss: 2.203253
Train Epoch: 2 [20480/50000 (41%)]	Loss: 2.191319
Train Epoch: 2 [25600/50000 (51%)]	Loss: 2.175657
Train Epoch: 2 [30720/50000 (61%)]	Loss: 2.170929
Train Epoch: 2 [35840/50000 (71%)]	Loss: 2.172102
Train Epoch: 2 [40960/50000 (82%)]	Loss: 2.101022
Train Epoch: 2 [46080/

### Hmm...

# Answer of the Model

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.C1 = nn.Conv2d(3, 6, kernel_size=5)
        self.S2 = nn.AvgPool2d(2, stride=2)
        self.C3 = nn.Conv2d(6, 16, kernel_size=5)
        self.S4 = nn.AvgPool2d(2, stride=2)
        self.C5 = nn.Conv2d(16, 120, kernel_size=5)
        self.F6 = nn.Linear(120, 84)
        self.OUTPUT = nn.Linear(84, 10)

    def forward(self, x):
        x = F.tanh(self.C1(x))
        x = F.tanh(self.S2(x))
        x = F.tanh(self.C3(x))
        x = F.tanh(self.S4(x))
        x = F.tanh(self.C5(x))
        x = x.view(-1, 120)
        x = F.tanh(self.F6(x))
        x = self.OUTPUT(x)
        return x